In [1]:
import pickle
import evaluate

from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    BartTokenizer,
    T5ForConditionalGeneration,
    T5Tokenizer,
)

## Models

### [t5-small-squad2-question-generation](https://huggingface.co/allenai/t5-small-squad2-question-generation)

In [2]:
def gen_t5_squad2(chunks):
    chunks_questions = {}

    model_name = "allenai/t5-small-squad2-question-generation"
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)
    for chunk in chunks:
        input_ids = tokenizer.encode(chunk["text"], return_tensors="pt")
        res = model.generate(input_ids)
        generated_questions = tokenizer.batch_decode(res, skip_special_tokens=True)
        chunks_questions[chunk["text"]] = generated_questions[0]

    return chunks_questions

### [Salesforce/discord_qg](https://huggingface.co/Salesforce/discord_qg)

> Uses the start word 'What' for generated questions

In [3]:
def gen_bart_discord(chunks):
    chunks_questions = {}

    qg_tokenizer = AutoTokenizer.from_pretrained("Salesforce/discord_qg")
    qg_model = AutoModelForSeq2SeqLM.from_pretrained("Salesforce/discord_qg")

    for chunk in chunks:
        encoder_ids = qg_tokenizer.batch_encode_plus(
            [chunk["text"]],
            add_special_tokens=True,
            padding=True,
            truncation=True,
            return_tensors="pt",
        )
        decoder_input_ids = qg_tokenizer.batch_encode_plus(
            ['What'],
            add_special_tokens=True,
            return_tensors="pt"
        )["input_ids"][:, :-1]
        model_output = qg_model.generate(
            **encoder_ids,
            decoder_input_ids=decoder_input_ids
        )
        generated_questions = qg_tokenizer.batch_decode(
            model_output, skip_special_tokens=True
        )

        chunks_questions[chunk["text"]] = generated_questions[0]

    return chunks_questions

### [bart-qg-nq-checkpoint](https://huggingface.co/McGill-NLP/bart-qg-nq-checkpoint)

In [4]:
def gen_bart_nq(chunks):
    chunks_questions = {}

    tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
    model = AutoModelForSeq2SeqLM.from_pretrained("McGill-NLP/bart-qg-nq-checkpoint")

    for chunk in chunks:
        inputs = tokenizer([chunk["text"]], return_tensors="pt")
        question_ids = model.generate(inputs["input_ids"])
        generated_questions = tokenizer.batch_decode(
            question_ids, skip_special_tokens=True
        )

        chunks_questions[chunk["text"]] = generated_questions[0]

    return chunks_questions

### [bart-eqg-question-generator](https://huggingface.co/voidful/bart-eqg-question-generator)

In [5]:
def gen_bart_eqg(chunks):
    chunks_questions = {}

    qg_tokenizer = AutoTokenizer.from_pretrained("voidful/bart-eqg-question-generator")
    qg_model = AutoModelForSeq2SeqLM.from_pretrained(
        "voidful/bart-eqg-question-generator"
    )

    for chunk in chunks:
        inputs = qg_tokenizer([chunk["text"]], return_tensors="pt")

        question_ids = qg_model.generate(inputs["input_ids"])
        generated_questions = qg_tokenizer.batch_decode(
            question_ids, skip_special_tokens=True
        )

        chunks_questions[chunk["text"]] = generated_questions[0]

    return chunks_questions

### [context-only-question-generator](https://huggingface.co/voidful/context-only-question-generator)

> Based on pretrained `bart-base` model

In [6]:
def gen_bart_unknown(chunks):
    chunks_questions = {}

    qg_tokenizer = AutoTokenizer.from_pretrained(
        "voidful/context-only-question-generator"
    )
    qg_model = AutoModelForSeq2SeqLM.from_pretrained(
        "voidful/context-only-question-generator"
    )

    for chunk in chunks:
        inputs = qg_tokenizer([chunk["text"]], return_tensors="pt")

        question_ids = qg_model.generate(inputs["input_ids"])
        generated_questions = qg_tokenizer.batch_decode(
            question_ids, skip_special_tokens=True
        )

        chunks_questions[chunk["text"]] = generated_questions[0]

    return chunks_questions

### Human generated questions

> Used as the reference questions for evaluation

In [7]:
# TODO: rewrite these questions w/o GPT
questions_list = [
    "What is the main topic of the lecture?",
    "What is the speaker's intention?",
    "Can you estimate the level of detail the lecture will cover regarding natural language processing?",
    "What is the speaker asking the audience to do?",
    "What is the primary focus of the discussion?",
    "What does the speaker inquire about regarding the estimate?",
    "What is the connection between smartphones and embedded AIs?",
    "What are the various subsystems mentioned that utilize AI in smartphones?",
    "Why is language considered a key element in interaction?",
    "What distinction is made between natural languages and formal languages like Python or C++?",
    "What fields contribute to natural language processing at its core?",
    "What are the two main subfields mentioned within NLP?",
    "What role does natural language understanding play in NLP?",
    "What does the speaker present as the motivation for natural language understanding?",
    "What types of insights can be gained from analyzing large amounts of text data?",
    "What are some common applications of natural language generation systems?",
    "What does the speaker present as the evolution of NLP systems over time?",
    "What are the main types of early NLP systems mentioned?",
    "What role did classical machine learning algorithms play in the development of NLP?",
    "What are the advancements mentioned that characterize the current state of NLP research?",
    "What are some common subfields of NLP?",
    "What is the significance of considering documents and corpora in the context of NLP?",
    "What are the primary steps in the NLP pre-processing pipeline?",
    "How does annotation contribute to the enrichment of tokens?",
    "What is the distinction between lemmatization and stemming in terms of processing words?",
    "Why is filtering important in the NLP pre-processing pipeline?",
    "What are n-grams, and why might they be valuable in NLP?",
    "What paper is recommended for reading?",
    "How does spelling correction in Microsoft Word relate to NLP concepts?",
    "What is the difference between lemmatization and stemming?",
    "How does the time-saving aspect influence the choice between lemmatization and stemming?",
    "How does filtering contribute to the efficiency of NLP models?",
    "How is the process of data analysis influenced by the need to save time in NLP?",
    "What is the role of dictionaries in lemmatization?",
    "Where can students find information about upcoming lab sessions and coursework on Moodle?",
]

def gen_reference_questions(chunks):
    chunks_questions = {}

    for i, chunk in enumerate(chunks):
        chunks_questions[chunk["text"]] = questions_list[i]

    return chunks_questions

## Chunking

In [8]:
with open("comp3074_lecture_2.pkl", "rb") as file:
    chunks = pickle.load(file)
    time_chunks = []
    current_sentence = ""
    start_timestamp = None
    min_duration = 60

    if not chunks["chunks"][-1]["timestamp"][1]:
        chunks["chunks"][-1]["timestamp"] = (
            chunks["chunks"][-1]["timestamp"][0],
            chunks["chunks"][-1]["timestamp"][0],
        )

    for chunk in chunks["chunks"]:
        text = chunk["text"]
        timestamp = chunk["timestamp"]

        if start_timestamp is None:
            # Start a new sentence
            start_timestamp = timestamp[0]

        current_sentence += text

        sentence_completed = text.strip()[-1] in ".!?" 
        time_elapsed = (timestamp[1] - start_timestamp) >= min_duration

        # TODO: tokenize to ensure below token limit for qg model
        # TODO: add overlap between chunks
        if sentence_completed and time_elapsed:
            time_chunks.append(
                {
                    "timestamp": (start_timestamp, timestamp[1]),
                    "text": current_sentence.strip(),
                }
            )
            current_sentence = ""
            start_timestamp = None

    if current_sentence.strip():
        time_chunks.append(
            {
                "timestamp": (start_timestamp, chunks["chunks"][-1]["timestamp"][1]),
                "text": current_sentence.strip(),
            }
        )

time_chunks

[{'timestamp': (0.0, 66.0),
  'text': "So in this lecture we're going to still be quite general. I'm going to talk about natural language processing and I'm going to talk in the first part I'm going to talk about what it is and in the second part I am going to oh good point in a second part I am going to talk about to introduce the concept of an NLP pipeline which is something that we'll be dealing with in the lab a little bit tomorrow and then from next Friday onwards. So first a quick overview of natural English processing. This is a topic which is usually reserved for like graduate courses in themselves so we're only even scrape really the surface of it. First a quick warm-up question and just take a minute and think can someone give me a quick estimate of how many AIs are in this room? Anyone throws out a guess?"},
 {'timestamp': (66.0, 128.0),
  'text': "I guess. Do you want individual phones or do you want like the types of hands? Well, that's the question, isn't it? Just as I ha

In [ ]:
models_questions = {
    "bart_unknown": gen_bart_unknown(time_chunks),
    "t5_squad2": gen_t5_squad2(time_chunks),
    "bart_discord": gen_bart_discord(time_chunks),
    "bart_nq": gen_bart_nq(time_chunks),
    "bart_eqg": gen_bart_eqg(time_chunks),
}

chunks_by_model = {}

for model, chunk_questions in models_questions.items():
    for chunk, question in chunk_questions.items():
        if chunk in chunks_by_model:
            chunks_by_model[chunk].update({model: question})
        else:
            chunks_by_model[chunk] = {model: question}

chunks_by_model

C:\Users\Omar\Documents\University\COMP4028\mmqg\.venv\Lib\site-packages\transformers\generation\utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


## Evaluation

> Using BERTScore

In [ ]:
def evaluate_contexts_all_metrics(chunks):
    bertscore = evaluate.load("bertscore")
    reference_chunks = gen_reference_questions(time_chunks)
    output = {}
    
    for context in chunks:
        references = [reference_chunks[context]]
        models = chunks[context]
        output[context] = {}

        for model in models:
            predictions = [models[model]]
            results = bertscore.compute(
                references=references, 
                predictions=predictions, 
                model_type="microsoft/deberta-xlarge-mnli"
            )
            output[context][model] = results
                

    return output


eval_scores = evaluate_contexts_all_metrics(chunks_by_model)
eval_scores

In [ ]:
from collections import defaultdict
import statistics

def calculate_avg_std_evaluation(chunks):
    precision_by_model = defaultdict(list)
    f1_by_model = defaultdict(list)
    recall_by_model = defaultdict(list)

    for context in chunks:
        models = chunks[context]
        for model in models:
            metrics = models[model]
            precision_by_model[model].append(metrics['precision'][0])
            f1_by_model[model].append(metrics['f1'][0])
            recall_by_model[model].append(metrics['recall'][0])
    
    stats_by_model = defaultdict(dict)
    for model in precision_by_model:
        precision_avg = sum(precision_by_model[model]) / len(precision_by_model[model])
        precision_std = statistics.stdev(precision_by_model[model])
        recall_avg = sum(recall_by_model[model]) / len(recall_by_model[model])
        recall_std = statistics.stdev(recall_by_model[model])
        f1_avg = sum(f1_by_model[model]) / len(f1_by_model[model])
        f1_std = statistics.stdev(f1_by_model[model])

        stats_by_model[model] = {
            'avg_precision': precision_avg,
            'std_precision': precision_std,
            'avg_recall': recall_avg,
            'std_recall': recall_std,
            'avg_f1': f1_avg,
            'std_f1': f1_std
        }
        
    return stats_by_model

average_eval_scores = calculate_avg_std_evaluation(eval_scores)
average_eval_scores

In [ ]:
sorted_models = sorted(average_eval_scores.items(), key=lambda x: x[1]['avg_f1'], reverse=True)

for model, stats in sorted_models:
    print(f"Model: {model}")
    print(f"F1 Score - Average and standard deviation: ({stats['avg_f1']:.3f} ± {stats['std_f1']:.3f})")
    print(f"Precision - Average and standard deviation: ({stats['avg_precision']:.3f} ± {stats['std_precision']:.3f})")
    print(f"Recall - Average and standard deviation: ({stats['avg_recall']:.3f} ± {stats['std_recall']:.3f})")
    print("-" * 60)